In [21]:
# define imports
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
#import PIL
import os, random

from glob import glob
from mpl_toolkits.axes_grid1 import ImageGrid
from IPython.display import display, Image
from IPython.core.display import HTML



import copy
#import cv2                

from io import open
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os, os.path, random
from PIL import Image
import requests
import shutil
import time

import torch
from torch.autograd import Variable
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms


%matplotlib inline


# my imports

from os import listdir
from os.path import isfile, join, abspath
from style_transfer.stylize import transfer


In [22]:
# All images are resized to 224x224 and normalized
# Only training images receive further augmentation

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),    
}


In [23]:
data_dir = 'dogImages_style_transfer'
print(os.path.join(data_dir, 'valid'))


# we create some dictionaries
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=5,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
print(dataset_sizes)
class_names = image_datasets['train'].classes
n_classes = len(class_names)

dogImages_style_transfer/valid
{'train': 59800, 'valid': 835, 'test': 836}


In [24]:
#import pandas as pd
import boto3
import sagemaker

In [25]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [26]:
import os

# this is the S3 data_dir name
data_dir = 'dogImages'
train_dir = 'train'

# set prefix, a descriptive name for a directory  
prefix = 'capstone-project'

# upload data for the first time; otherwise use the second expression (because the S3 upload takes some time)
#train_location = sagemaker_session.upload_data(os.path.join(data_dir), key_prefix=prefix)
train_location = 's3://{}/{}/'.format(bucket, prefix) 

In [27]:
# your import and estimator code, here
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

# note: this PyTorch estimator is specified with code that only
# uses the style transferred pictures, but no imgaug preprocessing!
source_dir = 'source_pytorch_style_transfer'

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir=source_dir,
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    #train_instance_type='ml.c4.xlarge',
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 10
                    })


In [28]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': train_location})

2019-08-06 07:17:23 Starting - Starting the training job...
2019-08-06 07:17:25 Starting - Launching requested ML instances......
2019-08-06 07:18:24 Starting - Preparing the instances for training......
2019-08-06 07:19:30 Downloading - Downloading input data................................................
2019-08-06 07:27:52 Training - Downloading the training image..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-06 07:28:06,957 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-06 07:28:06,981 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-06 07:28:06,982 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-08-06 07:28:07,231 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-08-06 07:28:07,231 sagemaker-containers INFO     Generating setup.cfg


Epoch: 1 #011Training Loss: 3.636029 #011Validation Loss: 1.318098
Epoch: 2 #011Training Loss: 2.686004 #011Validation Loss: 0.844991
Epoch: 3 #011Training Loss: 2.387923 #011Validation Loss: 0.685585
Epoch: 4 #011Training Loss: 2.232464 #011Validation Loss: 0.606484
Epoch: 5 #011Training Loss: 2.129745 #011Validation Loss: 0.561718
Epoch: 6 #011Training Loss: 2.060371 #011Validation Loss: 0.529383
Epoch: 7 #011Training Loss: 2.005694 #011Validation Loss: 0.505579
Epoch: 8 #011Training Loss: 1.954784 #011Validation Loss: 0.488833
Epoch: 9 #011Training Loss: 1.920151 #011Validation Loss: 0.476198
Epoch: 10 #011Training Loss: 1.887669 #011Validation Loss: 0.465194
2019-08-06 09:12:41,527 sagemaker-containers INFO     Reporting training SUCCESS

2019-08-06 09:12:45 Uploading - Uploading generated training model
2019-08-06 09:14:00 Completed - Training job completed
Billable seconds: 6871
CPU times: user 12.9 s, sys: 626 ms, total: 13.5 s
Wall time: 1h 56min 58s


In [29]:
%%time

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

print(estimator.model_data)

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir=source_dir)


s3://sagemaker-eu-central-1-505649883860/capstone-project/sagemaker-pytorch-2019-08-06-07-17-23-482/output/model.tar.gz
CPU times: user 17.6 ms, sys: 46 µs, total: 17.6 ms
Wall time: 71.1 ms


In [30]:
# ATTENTION: Please use the ml.t2.large instance here. Otherwise the deployment will fail because of resource constraints
# predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.large')



----------------------------------------------------------------------------------------------------!

In [31]:
def test_acc(test_set, predictor):
    ''' 
    This method calculates our accuracy metric. For this, it will send 
    batches of data to the endpoint and receive the predictions
    '''
    test_acc = 0.0
    running_corrects = 0

    for inputs, labels in test_set:
        print(labels)
        test_y_preds = predictor.predict(inputs)

        _, preds_tensor = torch.max(torch.from_numpy(test_y_preds), 1)
        preds = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())

        print(f"preds is {preds}")
    
        # compare the batch of predictions with the ground truth of labels
        running_corrects += torch.sum(torch.from_numpy(preds) == labels)

    test_acc = running_corrects.double() / len(test_set.dataset)

    print('Acc: {:.2f}%'.format(test_acc*100))

    return test_acc

use_cuda = torch.cuda.is_available()
testacc=test_acc(iter(dataloaders['test']), predictor)
print(testacc)

tensor([ 37,  49,  35, 128,  70])
preds is [ 37  59  35 128  70]
tensor([70, 93, 88, 81, 56])
preds is [70 93 88 81 56]
tensor([38, 37, 36, 12, 35])
preds is [ 38  37  36 126  35]
tensor([130,  84,  23,  21,  86])
preds is [130  84  23  21  86]
tensor([110,  39,   8,  94, 114])
preds is [110  39  87  94 114]
tensor([ 16,  30,  35,  81, 101])
preds is [ 16  30  35  81 101]
tensor([88, 96, 83, 78, 37])
preds is [88  2 83 94 37]
tensor([ 57,   5,  90, 127,  77])
preds is [ 57   5  90 127  80]
tensor([ 34, 100,  31,  18,  65])
preds is [ 95 100  31  18  65]
tensor([131,  14, 126,  92,  60])
preds is [131  14 126  25  60]
tensor([ 34,  64,  30,  54, 124])
preds is [ 34  64  30  54 124]
tensor([110,  21,   3,  20,  11])
preds is [110  21   3  20  11]
tensor([115,  91,  32,   7,  15])
preds is [115  91  32   7   6]
tensor([ 67,   7,  49, 110,  96])
preds is [ 67   7  49 110 124]
tensor([121,   6,  87,  14, 122])
preds is [121   6   8  14 122]
tensor([56, 98,  4, 94, 31])
preds is [56 98  4 78

preds is [ 55   0 124  86  14]
tensor([14, 47, 30, 99, 95])
preds is [ 14 107  30 103  46]
tensor([ 22, 111,  48, 114, 100])
preds is [ 22 111  48 114 100]
tensor([ 62,  88,  44,  57, 119])
preds is [ 62  88  44  57 119]
tensor([ 22,  78, 100,   0, 129])
preds is [ 22  78 100  98 129]
tensor([ 15,  80,  67, 131,  12])
preds is [ 15  80  67 131  12]
tensor([22, 17, 22, 47, 82])
preds is [22 17 22 47 82]
tensor([73, 11, 16, 75, 28])
preds is [73 11 16 75 28]
tensor([59, 93, 52, 49, 69])
preds is [59 93 52 49 69]
tensor([97,  3, 45, 23,  0])
preds is [97 42 45 23  0]
tensor([53, 57, 29,  3, 83])
preds is [ 53  57  29   4 114]
tensor([54, 78, 52, 67, 68])
preds is [54 94 52 67 68]
tensor([ 54, 116,  75,  13,  52])
preds is [ 54 116  75  13 129]
tensor([56, 58,  3, 65, 81])
preds is [56 69  3 65 81]
tensor([ 48,  84,  86,  31, 132])
preds is [123  84  86  31 132]
tensor([ 64,  94,  30, 122, 117])
preds is [ 64  94  53 122 117]
tensor([81, 89, 45,  9, 99])
preds is [124  89  76   9  99]
tens

## The baseline model

Finally we want to see how our baseline model keeps up. For this, we first need to upload the original dogImages (no style transferred images includes)

In [6]:
import os

# this is the S3 data_dir name
data_dir = 'dogImages'

# set prefix, a descriptive name for a directory  
prefix = 'capstone-project-orig-data'

# upload data for the first time; otherwise use the second expression (because the S3 upload takes some time)
#train_location_orig_data = sagemaker_session.upload_data(os.path.join(data_dir), key_prefix=prefix)
train_location_orig_data = 's3://{}/{}/'.format(bucket, prefix) 

The estimator itself and the deployment is exactly the same. We only change the `source_dir`, that specifies our `train` and `predict` file. In case of our baseline we want to keep everything as simple as possible - that is why there is no advanced image augmenting included with the `imgaug` library. For more details have a look at `source_pytorch_style_transfer/train.py`

In [7]:
# your import and estimator code, here
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

# note: this PyTorch estimator is specified with code that only
# uses the style transferred pictures, but no imgaug preprocessing!
source_dir = 'source_pytorch_style_transfer'

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir=source_dir,
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    #train_instance_type='ml.c4.xlarge',
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 10
                    })



In [8]:
%%time
# Train your estimator on S3 training data
estimator.fit({'train': train_location_orig_data})

2019-08-06 06:15:31 Starting - Starting the training job...
2019-08-06 06:15:34 Starting - Launching requested ML instances......
2019-08-06 06:16:32 Starting - Preparing the instances for training......
2019-08-06 06:17:48 Downloading - Downloading input data.........
2019-08-06 06:19:28 Training - Downloading the training image..

2019-08-06 06:19:46 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-06 06:19:48,128 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-06 06:19:48,153 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-06 06:19:48,460 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-08-06 06:19:48,740 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-08-06 06:19:48,740 sa

Epoch: 1 #011Training Loss: 3.712942 #011Validation Loss: 2.561787
Epoch: 2 #011Training Loss: 2.093467 #011Validation Loss: 1.663501
Epoch: 3 #011Training Loss: 1.476688 #011Validation Loss: 1.265527
Epoch: 4 #011Training Loss: 1.181037 #011Validation Loss: 1.050675
Epoch: 5 #011Training Loss: 1.011723 #011Validation Loss: 0.915356
Epoch: 6 #011Training Loss: 0.890234 #011Validation Loss: 0.823472
Epoch: 7 #011Training Loss: 0.804099 #011Validation Loss: 0.756152
Epoch: 8 #011Training Loss: 0.736964 #011Validation Loss: 0.703578
Epoch: 9 #011Training Loss: 0.688688 #011Validation Loss: 0.662970
Epoch: 10 #011Training Loss: 0.645880 #011Validation Loss: 0.631162
2019-08-06 06:34:39,087 sagemaker-containers INFO     Reporting training SUCCESS

2019-08-06 06:35:15 Uploading - Uploading generated training model
2019-08-06 06:52:05 Completed - Training job completed
Billable seconds: 2057
CPU times: user 4.05 s, sys: 129 ms, total: 4.18 s
Wall time: 37min 1s


In [11]:
%%time

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

print(estimator.model_data)

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir=source_dir)

s3://sagemaker-eu-central-1-505649883860/capstone-project-orig-data/sagemaker-pytorch-2019-08-06-06-15-30-723/output/model.tar.gz
CPU times: user 13.6 ms, sys: 3.91 ms, total: 17.5 ms
Wall time: 119 ms


In [12]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.large')

-------------------------------------------------------------------------------------!

In [13]:
data_dir = 'dogImages'
print(os.path.join(data_dir, 'valid'))


# we create some dictionaries
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=5,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}

dogImages/valid


In [16]:
testacc=test_acc(iter(dataloaders['test']), predictor)
print(testacc)

tensor([ 43, 110,  45,  38,  46])
preds is [ 43 110  76  38  46]
tensor([ 84,  13,   0, 111,  64])
preds is [ 84  42   0 111  64]
tensor([ 50,  30, 114,  40,  15])
preds is [ 50  30 114  40  55]
tensor([ 67, 103,  58, 130,  29])
preds is [ 67 103  58  72  29]
tensor([113,  95,  50,  59,  62])
preds is [113  46  50  59  62]
tensor([ 21,  80,  35, 126,   7])
preds is [ 21  80  35 126   7]
tensor([35, 51, 53, 38,  3])
preds is [35 51 53 38  3]
tensor([  1,  89, 116,  42, 124])
preds is [  1 119 116 107 124]
tensor([11, 12, 32, 11, 27])
preds is [111  12  32  11 121]
tensor([ 55, 122,  61,  41,  85])
preds is [ 55 122  61  41  85]
tensor([118,  81,  19,  91,  90])
preds is [118  81  19  91  90]
tensor([71, 85, 88, 55, 40])
preds is [71 85 88 69 43]
tensor([81, 12, 13, 53, 96])
preds is [ 81 126  13  53 124]
tensor([ 70,  92,  67, 117,  79])
preds is [ 70 124  67 117  79]
tensor([110,  52,  78, 116,   4])
preds is [110  60  78 116   4]
tensor([ 21, 123,  99,  63,  81])
preds is [ 21 123  99

preds is [ 60 118   4 111  89]
tensor([ 95, 108,  39,  12,  26])
preds is [ 95 108  39  12  26]
tensor([117,  41,  88,  72, 122])
preds is [117  41  16 130 122]
tensor([ 27,   9, 117, 104,  76])
preds is [ 27   9 117 104  76]
tensor([ 23,  71, 101,  59,   9])
preds is [ 23  71 101  59   9]
tensor([86,  4, 16, 77, 27])
preds is [86  4 16 80 27]
tensor([90, 95, 83, 65, 49])
preds is [90 95 83 65 59]
tensor([89,  6, 90, 97, 79])
preds is [ 89   6  90  97 121]
tensor([ 23,  44,  63,  45, 129])
preds is [ 23  44  45  45 129]
tensor([ 21, 122, 128,  60,  88])
preds is [ 21 122 128  62  88]
tensor([106,  15, 125,  45,  56])
preds is [106  15 125  45  56]
tensor([31,  6, 17, 96, 85])
preds is [31  6 17  2 85]
tensor([109, 129,  80,  33,  88])
preds is [127 129  89  33  88]
tensor([ 48,  48,  91, 101,  24])
preds is [ 48  48  91 101  24]
tensor([25, 83, 58, 35, 52])
preds is [25 83 69 35 52]
tensor([111, 102,  56,   7,  97])
preds is [111 102  56   7  97]
tensor([50, 62, 23, 42, 14])
preds is [

In [19]:
print(source_dir)
print(train_location_orig_data)

source_pytorch_style_transfer
s3://sagemaker-eu-central-1-505649883860/capstone-project-orig-data/
